In [3]:
import csv
from itertools import product
import numpy as np
import os
import pandas as pd
import requests
import time
from urllib2 import urlopen
import urllib2

In [88]:
#open files
df_raw_herb_hmdb_pair = pd.read_csv('/home/share/aschoi/data/COCONUT_Herbal/5_Similarity/7_sorted_score_vector_except_1(descending)_top_10_percent_no_score.csv', header=None)
df_hmdb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/4_HMDB/HMDB_KEGG.tsv')
df_herb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/2-5_herb_pubchem.tsv')
df_kegg_rpair = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/1_KEGG_rpair/KEGG_Rpair.txt')
df_orthology = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/5_predicted_orthology.tsv')

In [89]:
# extract elements of top percentage.
#------------------------------------------------------------------------------
WHOLE_LENGTH = 646647053
percentage = input('Enter the percentage number (0,10] : ')
percentage_idx = int(WHOLE_LENGTH * percentage / 100)
df_top_herb_hmdb_pair = df_raw_herb_hmdb_pair[:percentage_idx]
df_top_herb_hmdb_pair = df_top_herb_hmdb_pair.rename(columns={0:'compID', 1:'Accession'})

Enter the percentage number (0,10] : 0.1


In [34]:
print len(df_top_herb_hmdb_pair)
df_top_herb_hmdb_pair.head()

646647


,compID,Accession
0,80303,HMDB37761
1,69881,HMDB02174
2,69881,HMDB02308
3,69881,HMDB03458
4,69881,HMDB06316


In [35]:
#top herb id, top hmdb id
#------------------------------------------------------------------------------
s_top_herb = df_top_herb_hmdb_pair['compID']
s_top_hmdb = df_top_herb_hmdb_pair['Accession']
print("    1. Extract the ids of top herbal and top HMDB compounds.")
#for i in top_herb_hmdb_pair:
#    top_herb.append(int(i[0]))
#    top_hmdb.append(i[1].rstrip('\r\n'))

    1. Extract the ids of top herbal and top HMDB compounds.


In [36]:
print len(s_top_herb)
print len(s_top_hmdb)

646647
646647


In [37]:
# Dataframe of top_herb_hmdb_pair
#df_top_herb_hmdb_pair = pd.DataFrame({'Accession':top_hmdb, 'compID' :top_herb})

In [38]:
# Find the KEGG ID of top_hmdb.

# The flag is used to find the KEGG ID of top_similar_hmdb.
flag_find_keggid_of_top_hmdb = df_hmdb['Accession'].isin(s_top_hmdb)

# This dataframe represents the pair of top hmdb id and its KEGG ID.
# This information is used when find the substrate of KEGG rpair which correspond to the top_hmdb.
df_top_hmdb_keggid_pair = df_hmdb[flag_find_keggid_of_top_hmdb][['Accession', 'KEGG_ID']]

In [39]:
print len(df_top_hmdb_keggid_pair)
df_top_hmdb_keggid_pair.head()

1130


,Accession,KEGG_ID
0,HMDB00002,C00986
1,HMDB00005,C00109
2,HMDB00010,C05299
3,HMDB00011,C01089
6,HMDB00016,C03205


In [40]:
# Find the PubChem ID of top_herb.

# The flag is used to find the PubChem ID of top_herb.
flag_find_pubchemid_of_top_herb = df_herb['compID'].isin(s_top_herb)

# This dataframe represents the pair of top herb id and its PubChem ID.
# Using this information, we can predict the orphan herbs' ontology.
# One pubchem id of herb becomes the input substrate of KEGG E-zyme2 tool.
df_top_herb_pubchemid_pair = df_herb[flag_find_pubchemid_of_top_herb][['compID', 'pubchemID']]

In [41]:
print len(df_top_herb_pubchemid_pair)
df_top_herb_pubchemid_pair.head()

6826


,compID,pubchemID
6,37772,124039
7,33032,177786
8,33034,12004512
9,33179,88810
11,32814,313075


In [42]:
# Find the rpairs whose substrate is same to the top_hmdb.
# The pair is represented by only KEGG ID. (NOT hmdb_id.)

flag_find_rpair_of_top_hmdb_sub = df_kegg_rpair['Compound1'].isin(df_top_hmdb_keggid_pair['KEGG_ID']) 

# This dataframe consist of KEGG ID.
df_rpair_of_top_hmdb_sub_k = df_kegg_rpair[flag_find_rpair_of_top_hmdb_sub]

In [43]:
print len(df_rpair_of_top_hmdb_sub_k)
df_rpair_of_top_hmdb_sub_k.head()

3317


,Compound1,Compound2
1,C00003,C00004
2,C00002,C00008
6,C00002,C00020
7,C00002,C00013
8,C00002,C00009


In [44]:
# Find the part of top_hmdb. 
# It have KEGG ID which corresponds to the substrate of KEGG rpair.
# Why need this process? 
# Because we don't have the pair of the kegg_id version substrate(although it corresponds to the top_hmdb!) and top_hmdb id.

flag_find_hmdbid_of_top_hmdb_sub = df_top_hmdb_keggid_pair['KEGG_ID'].isin(df_rpair_of_top_hmdb_sub_k['Compound1'])
df_top_hmdb_of_sub = df_top_hmdb_keggid_pair[flag_find_hmdbid_of_top_hmdb_sub]

In [45]:
print len(df_top_hmdb_of_sub)
df_top_hmdb_of_sub.head()

543


,Accession,KEGG_ID
0,HMDB00002,C00986
1,HMDB00005,C00109
2,HMDB00010,C05299
3,HMDB00011,C01089
6,HMDB00016,C03205


In [46]:
# Find the pair of top_hmdb and top_herb.
# The top_hmdb is the substrate of the KEGG rpair.
# The top_herb is the similar herbal compounds of the top_hmdb.

flag_find_sim_herb_of_hmdb_sub = df_top_herb_hmdb_pair['Accession'].isin(df_top_hmdb_of_sub['Accession'])
df_sim_herb_of_hmdb_sub = df_top_herb_hmdb_pair[flag_find_sim_herb_of_hmdb_sub]

# Find the pubchem id of the herbs which is similar to the hmdb compounds. These hmdb compounds are the substrate of rpair. 
flag_find_pubchemid_of_sim_herb =df_top_herb_pubchemid_pair['compID'].isin(df_sim_herb_of_hmdb_sub['compID'])
df_sim_herb_pubchem_id_pair = df_top_herb_pubchemid_pair[flag_find_pubchemid_of_sim_herb]

df_candidate_sub_herb = pd.merge(df_top_hmdb_of_sub, df_sim_herb_of_hmdb_sub, how = 'inner', on = ['Accession', 'Accession'])
df_candidate_sub_herb_pubchem = pd.merge(df_candidate_sub_herb, df_sim_herb_pubchem_id_pair, how='inner', on=['compID', 'compID'])


In [47]:
print len(df_candidate_sub_herb_pubchem)
df_candidate_sub_herb_pubchem.head()

8196


,Accession,KEGG_ID,compID,pubchemID
0,HMDB00002,C00986,65801,3483736
1,HMDB00005,C00109,70898,450639
2,HMDB00011,C01089,70898,450639
3,HMDB00019,C00141,70898,450639
4,HMDB00060,C00164,70898,450639


In [48]:
# Make the pair of herb(used to be substrate) and product.
# If the pair will exist in the prepared orthology table, just show the predicted_result.
# Unless the pair will not exist in the prepared table, we have to crawl the predicted reusult from KEGG E-zyme2 webpage.

from itertools import product
s_candidate_keggid = df_candidate_sub_herb_pubchem['KEGG_ID'].drop_duplicates()
df_query = pd.DataFrame()
for i in s_candidate_keggid:
    df_candidate_sub_part = df_candidate_sub_herb_pubchem[df_candidate_sub_herb_pubchem.KEGG_ID == i]
    df_candidate_pro_part = df_rpair_of_top_hmdb_sub_k[df_rpair_of_top_hmdb_sub_k.Compound1 == i]
    s_candidate_sub_part = df_candidate_sub_part['pubchemID']
    s_candidate_pro_part = df_candidate_pro_part['Compound2']
    temp_df = pd.DataFrame(list(product(s_candidate_sub_part, s_candidate_pro_part)), columns = ['substrate', 'product'])
    df_query = df_query.append(temp_df)
df_query = df_query.drop_duplicates()
df_query.index = range(0, len(df_query))

In [49]:
print "       inner query_list : len(df_candidate_sub_herb_pubchem, df_rpair_of_top_hmdb_sub_k) %d, %d" % (len(df_candidate_sub_herb_pubchem), len(df_rpair_of_top_hmdb_sub_k))

       inner query_list : len(df_candidate_sub_herb_pubchem, df_rpair_of_top_hmdb_sub_k) 8196, 3317


In [50]:
len(s_candidate_keggid)

491

In [51]:
print len(df_query)
df_query.head()

26527


,substrate,product
0,3483736,C05665
1,3483736,C03375
2,3483736,C03283
3,3483736,C15853
4,3483736,C01029


In [42]:
#다시 시작시 파일명 바꾸기
with open("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/predicted_orthology_test_1.tsv", "w") as resultFile:
    features = ['substrate', 'product', 'predicted_orthology', 'predicted_score']
    resultTsv = csv.DictWriter(resultFile, features, dialect=csv.excel_tab)
    resultTsv.writeheader()
    #predicted_result = pd.DataFrame()
    cnt = 0
    url_base = 'http://rest.genome.jp/ezyme/'
    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
    for index, row in df_query[cnt:].iterrows():
        sub = int(row['substrate'])
        pro = row['product']
        url = url_base + 'CID'+str(sub) +'/' + pro
        
        MAX_ATTEMPTS = 8
        for attempt in range(MAX_ATTEMPTS):
            try:
                f = urlopen(url)
            except urllib2.HTTPError, e:
                if e.code == 404:
                    print 'HTTPError 404 not found %d : ' % cnt + time.strftime('%a %H:%M:%S')
                    print url
                    cnt = cnt + 1
                    break
                else:
                    print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                    raise
            except urllib2.URLError, e:
                if e.args[0][0] == 110: # connection time out
                    sleep_secs = attempt ** 2
                    print e
                    print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    print url
                    print '    Retrying in %d seconds...' % sleep_secs
                    time.sleep(sleep_secs)
                    continue
                else:
                    print e
                    print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    raise
            else:          
                all_data = f.readlines()
                data = all_data[1].split('\t')
                predicted_orthology = data[1]
                predicted_score = data[0]
                resultTsv.writerow({'substrate':sub, 'product':pro, 'predicted_orthology' : predicted_orthology, 'predicted_score' : predicted_score})     
                #temp_df = pd.DataFrame([[sub, pro, predicted_orthology, predicted_score]], columns = ['substrate', 'product', 'predicted_orthology', 'predicted_score'])
                #predicted_result2 = predicted_result2.append(temp_df)
                cnt = cnt + 1
                break
        else:
            print 'we failed to reconnect 8 times in %s' % url 
        
        if cnt % 1000 == 0 :
            print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

0 : Mon 21:33:51
HTTPError 404 not found 198 : Mon 21:51:53
http://rest.genome.jp/ezyme/CID450639/C06201
HTTPError 404 not found 199 : Mon 21:51:57
http://rest.genome.jp/ezyme/CID450639/C00334
HTTPError 404 not found 200 : Mon 21:51:59
http://rest.genome.jp/ezyme/CID450639/C01215
HTTPError 404 not found 201 : Mon 21:52:02
http://rest.genome.jp/ezyme/CID450639/C05715
HTTPError 404 not found 202 : Mon 21:52:04
http://rest.genome.jp/ezyme/CID450639/C00755
HTTPError 404 not found 203 : Mon 21:52:07
http://rest.genome.jp/ezyme/CID450639/C19631
HTTPError 404 not found 287 : Mon 22:02:05
http://rest.genome.jp/ezyme/CID5318189/C04108
HTTPError 404 not found 325 : Mon 22:06:31
http://rest.genome.jp/ezyme/CID44559634/C05485
HTTPError 404 not found 347 : Mon 22:09:38
http://rest.genome.jp/ezyme/CID5320506/C04108
HTTPError 404 not found 436 : Mon 22:20:08
http://rest.genome.jp/ezyme/CID5318188/C05485
1000 : Mon 23:22:46
HTTPError 404 not found 1607 : Tue 00:33:03
http://rest.genome.jp/ezyme/CID167

In [56]:
f_http404 = open("/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/http404.txt", 'r')
http404 = list()
while True:
    line = f_http404.readline()
    if not line: break
    http404.append(line.rstrip('\r\n'))
f_http404.close()
        

In [59]:
http404[0]

'http://rest.genome.jp/ezyme/CID10030799/C01798'

In [58]:
http404[0].split('/')

['http:', '', 'rest.genome.jp', 'ezyme', 'CID10030799', 'C01798']

In [66]:
#다시 시작시 파일명 바꾸기
with open("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/predicted_orthology_test_2_error.tsv", "w") as resultFile:
    features = ['substrate', 'product', 'predicted_orthology', 'predicted_score']
    resultTsv = csv.DictWriter(resultFile, features, dialect=csv.excel_tab)
    resultTsv.writeheader()
    #predicted_result = pd.DataFrame()
    cnt = 0
    #url_base = 'http://rest.genome.jp/ezyme/'
    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
    #for index, row in df_query[cnt:].iterrows():
        #sub = int(row['substrate'])
        #pro = row['product']
        #url = url_base + 'CID'+str(sub) +'/' + pro
    for url in http404[cnt:] :
        MAX_ATTEMPTS = 8
        for attempt in range(MAX_ATTEMPTS):
            try:
                f = urlopen(url)
            except urllib2.HTTPError, e:
                if e.code == 404:
                    print 'HTTPError 404 not found %d : ' % cnt + time.strftime('%a %H:%M:%S')
                    print url
                    cnt = cnt + 1
                    break
                else:
                    print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                    raise
            except urllib2.URLError, e:
                if e.args[0][0] == 110: # connection time out
                    sleep_secs = attempt ** 2
                    print e
                    print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    print url
                    print '    Retrying in %d seconds...' % sleep_secs
                    time.sleep(sleep_secs)
                    continue
                else:
                    print e
                    print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    raise
            else:
                sub = url.split('/')[4]
                pro = url.split('/')[5]
                all_data = f.readlines()
                data = all_data[1].split('\t')
                predicted_orthology = data[1]
                predicted_score = data[0]
                resultTsv.writerow({'substrate':sub, 'product':pro, 'predicted_orthology' : predicted_orthology, 'predicted_score' : predicted_score})     
                #temp_df = pd.DataFrame([[sub, pro, predicted_orthology, predicted_score]], columns = ['substrate', 'product', 'predicted_orthology', 'predicted_score'])
                #predicted_result2 = predicted_result2.append(temp_df)
                cnt = cnt + 1
                break
        else:
            print 'we failed to reconnect 8 times in %s' % url 
        
        if cnt % 1000 == 0 :
            print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

0 : Tue 16:33:07
HTTPError 404 not found 0 : Tue 16:33:13
http://rest.genome.jp/ezyme/CID10030799/C01798
HTTPError 404 not found 1 : Tue 16:33:19
http://rest.genome.jp/ezyme/CID10030799/C05713
HTTPError 404 not found 2 : Tue 16:33:23
http://rest.genome.jp/ezyme/CID10080695/C08300
HTTPError 404 not found 3 : Tue 16:33:27
http://rest.genome.jp/ezyme/CID10080695/C16719
HTTPError 404 not found 4 : Tue 16:33:31
http://rest.genome.jp/ezyme/CID101/C03067
HTTPError 404 not found 5 : Tue 16:33:35
http://rest.genome.jp/ezyme/CID10123885/C00029
HTTPError 404 not found 6 : Tue 16:33:38
http://rest.genome.jp/ezyme/CID10123885/C00043
HTTPError 404 not found 7 : Tue 16:33:42
http://rest.genome.jp/ezyme/CID10123885/C00052
HTTPError 404 not found 8 : Tue 16:33:46
http://rest.genome.jp/ezyme/CID10123885/C00075
HTTPError 404 not found 9 : Tue 16:33:49
http://rest.genome.jp/ezyme/CID10123885/C00167
HTTPError 404 not found 10 : Tue 16:33:53
http://rest.genome.jp/ezyme/CID10123885/C00190
HTTPError 404 not f

In [67]:
print(len(df_orthology))
df_orthology.head()

26204


,substrate,product,predicted_orthology,predicted_score
0,3483736,C05665,K00276,0.381
1,3483736,C03375,K01844,0.053
2,3483736,C03283,K01580,0.736
3,3483736,C15853,K00632,0.106
4,3483736,C01029,K01844,0.044


In [68]:
df_test = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/predicted_orthology_test_1.tsv")
df_test_error = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/predicted_orthology_test_2_error.tsv")

In [80]:
frames = [df_test, df_test_error]
df = pd.concat(frames)
df.index = range(0, len(df))
print(len(df))
df.head()

26199


,substrate,product,predicted_orthology,predicted_score
0,3483736,C05665,K00276,0.381
1,3483736,C03375,K01844,0.053
2,3483736,C03283,K01580,0.736
3,3483736,C15853,K07513,0.106
4,3483736,C01029,K01844,0.044


In [87]:
df.to_csv("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/5_predicted_orthology.tsv", sep='\t', index=False)

In [ ]:
for index, row in df_query[cnt:].iterrows():
        sub = int(row['substrate'])
        pro = row['product']
        url = url_base + 'CID'+str(sub) +'/' + pro
    

        for attempt in range(MAX_ATTEMPTS):
                try:
                    f = urlopen(url)
                except urllib2.HTTPError, e:
                    if e.code == 404:
                        print 'HTTPError 404 not found %d : ' % cnt + time.strftime('%a %H:%M:%S')
                        print url
                        cnt = cnt + 1
                        break
                    else:
                        print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                        raise
                except urllib2.URLError, e:
                    if e.code == 110: # connection time out
                        sleep_secs = attempt ** 2
                        print 'ERROR: %s. cnt = %d, error time : ' % (e, cnt) + time.strftime('%a %H:%M:%S')
                        print url
                        print 'Retrying in %d seconds...' % sleep_secs
                        time.sleep(sleep_secs)
                        continue
                    else:
                        print 'e.code : %d, error message : %s, url : %s, cnt : d' % (e.code, e, url, cnt)
                        raise
                else:          
                    all_data = f.readlines()
                    data = all_data[1].split('\t')
                    predicted_orthology = data[1]
                    predicted_score = data[0]
                    resultTsv.writerow({'substrate':sub, 'product':pro, 'predicted_orthology' : predicted_orthology, 'predicted_score' : predicted_score})     
                    #temp_df = pd.DataFrame([[sub, pro, predicted_orthology, predicted_score]], columns = ['substrate', 'product', 'predicted_orthology', 'predicted_score'])
                    #predicted_result2 = predicted_result2.append(temp_df)
                    cnt = cnt + 1
                    break
        else:
            print 'we failed to reconnect 8 times in %s' % url 
        
        if cnt % 1000 == 0 :
            print '%d : '% cnt + time.strftime('%a %H:%M:%S')

In [37]:
cnt = 0
url_list = ['http://rest.genome.jp/ezyme/CID104745/C17238', 'http://rest.genome.jp/ezyme/CID5281127/C14825',
            'http://rest.genome.jp/ezyme/CID104745/C20258', 'http://rest.genome.jp/ezyme/CID4409936/C07086',
           'http://rest.genome.jp/ezyme/CID11993858/C03758']
temp = []
for url in url_list:
    try:
        a = urlopen(url)
        aa = a.readlines()
        aaa = aa[1].split('\t')
        temp.append((url,aaa[1]))
        cnt = cnt + 1
    except urllib2.HTTPError, e:
        if e.code == 404:
            print"error"
            a = urlopen(url)
            aa = a.readlines()
            aaa = aa[1].split('\t')
            temp.append((url,aaa[1]))
            cnt = cnt + 1
            continue

error


HTTPError: HTTP Error 404: Not Found

In [29]:
url

'http://rest.genome.jp/ezyme/CID104745/C17238'

In [33]:
a

<addinfourl at 140605074756672 whose fp = <socket._fileobject object at 0x7fe12b855ad0>>

In [35]:
temp

[('http://rest.genome.jp/ezyme/CID104745/C17238', 'K11819'),
 ('http://rest.genome.jp/ezyme/CID5281127/C14825', 'K07418'),
 ('http://rest.genome.jp/ezyme/CID104745/C20258', 'K01714')]

In [36]:
cnt

3